# Gemini 챗봇 만들기 — 단계별 튜토리얼

이 노트북은 Google Gemini API와 Gradio를 사용해서  
**나만의 AI 챗봇**을 만드는 과정을 단계별로 설명합니다.

---

## 목차
1. 라이브러리 설치
2. Gemini API 기본 사용법
3. 대화 이어가기 (채팅 모드)
4. 시스템 프롬프트 설정
5. Google Search 연동
6. Gradio로 웹 UI 만들기
7. 대화 저장
8. 완성 코드
9. Hugging Face에 배포하기

---
## 1단계: 라이브러리 설치

두 개의 라이브러리만 설치하면 됩니다:
- `google-genai`: Google Gemini API 사용 (새 SDK)
- `gradio`: 웹 UI 만들기

In [ ]:
# 라이브러리 설치 (최초 1회만 실행)
!pip install google-genai gradio

In [ ]:
# 라이브러리 불러오기
from google import genai
from google.genai import types
import gradio as gr
import json
import os
import datetime

print("라이브러리 로드 완료!")

---
## 2단계: Gemini API 기본 사용법

Gemini에게 질문을 보내고 답변을 받는 가장 기본적인 코드입니다.  
API Key는 https://aistudio.google.com/apikey 에서 발급받으세요.

In [ ]:
# 여기에 본인의 API Key를 입력하세요
MY_API_KEY = "여기에_API키_입력"

# 클라이언트 생성
client = genai.Client(api_key=MY_API_KEY)

# 질문 보내기
response = client.models.generate_content(
    model="gemini-2.5-flash-lite",
    contents="안녕! 자기소개 해줘."
)

# 답변 출력
print(response.text)

### 핵심 정리

```python
client = genai.Client(api_key=키)           # 1) 클라이언트 생성
response = client.models.generate_content(   # 2) 질문 보내기
    model="gemini-2.5-flash-lite",
    contents="질문"
)
print(response.text)                         # 3) 답변 받기
```

이 3줄이 Gemini API의 전부입니다!

---
## 3단계: 대화 이어가기 (채팅 모드)

`generate_content()`는 매번 독립적인 질문입니다.  
**대화를 이어가려면** `start_chat()`을 사용합니다.

In [ ]:
# 채팅: contents 리스트로 대화를 이어갑니다

contents = []

# 첫 번째 메시지
contents.append(types.Content(role="user", parts=[types.Part(text="내 이름은 지민이야. 기억해줘.")]))

response1 = client.models.generate_content(
    model="gemini-2.5-flash-lite",
    contents=contents,
)
print("AI:", response1.text)

# AI 응답을 기록에 추가
contents.append(types.Content(role="model", parts=[types.Part(text=response1.text)]))

print("---")

# 두 번째 메시지 (이전 대화를 기억함!)
contents.append(types.Content(role="user", parts=[types.Part(text="내 이름이 뭐라고 했지?")]))

response2 = client.models.generate_content(
    model="gemini-2.5-flash-lite",
    contents=contents,
)
print("AI:", response2.text)

### 핵심: 단일 질문 vs 대화 모드

| 방식 | 대화 기억 | 용도 |
|------|----------|------|
| `contents="질문"` (문자열) | 없음 | 일회성 질문 |
| `contents=[Content(...), ...]` (리스트) | 있음 | 대화형 챗봇 |

contents 리스트에 이전 대화를 누적하면 맥락을 기억합니다!

---
## 4단계: 시스템 프롬프트 설정

시스템 프롬프트로 AI의 **역할과 성격**을 지정할 수 있습니다.

In [ ]:
# 시스템 프롬프트로 AI 역할 설정
response = client.models.generate_content(
    model="gemini-2.5-flash-lite",
    contents="프로그래밍이 뭐야?",
    config=types.GenerateContentConfig(
        system_instruction="당신은 요리 전문가입니다. 모든 질문에 요리 비유를 사용해서 답변하세요."
    ),
)

print(response.text)

---
## 5단계: Google Search 연동

Gemini에게 **실시간 인터넷 검색** 능력을 부여할 수 있습니다.  
`tools` 파라미터에 `GoogleSearch`를 추가하면 됩니다.

In [ ]:
# Google Search가 가능한 설정
today = datetime.date.today().strftime("%Y-%m-%d")

response = client.models.generate_content(
    model="gemini-2.5-flash-lite",
    contents="오늘 서울 날씨 어때?",
    config=types.GenerateContentConfig(
        system_instruction=f"오늘 날짜는 {today}입니다. 최신 정보를 기준으로 답변하세요.",
        tools=[types.Tool(google_search=types.GoogleSearch())],  # 이 한 줄만 추가!
    ),
)

print(response.text)

### 💡 Google Search는 언제 유용할까?

- 오늘 뉴스, 날씨 등 **실시간 정보**가 필요할 때
- AI가 학습하지 못한 **최신 정보**를 물어볼 때
- 특정 웹사이트의 **사실 확인**이 필요할 때

---
## 6단계: Gradio로 웹 UI 만들기

지금까지 배운 것을 **웹 페이지로 만들어** 봅시다.  
Gradio를 사용하면 Python 코드만으로 웹 UI를 만들 수 있습니다.

### 6-1. Gradio 기초: 간단한 챗봇

In [ ]:
# 가장 간단한 Gradio 챗봇

def simple_chat(user_message, chat_history):
    """사용자 메시지를 받아서 AI 응답을 반환"""
    client = genai.Client(api_key=MY_API_KEY)
    response = client.models.generate_content(
        model="gemini-2.5-flash-lite",
        contents=user_message,
    )

    chat_history.append({"role": "user", "content": user_message})
    chat_history.append({"role": "assistant", "content": response.text})
    return "", chat_history

# UI 구성
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="메시지를 입력하세요")
    msg.submit(fn=simple_chat, inputs=[msg, chatbot], outputs=[msg, chatbot])

demo.launch()

### Gradio 핵심 개념

```python
with gr.Blocks() as demo:       # 1) 웹 페이지 틀 만들기
    chatbot = gr.Chatbot()       # 2) 채팅 창 배치
    msg = gr.Textbox()           # 3) 입력 창 배치
    msg.submit(fn=함수, ...)     # 4) Enter 누르면 함수 실행

demo.launch()                    # 5) 웹 서버 시작!
```

Gradio = **컴포넌트 배치** + **이벤트 연결**, 이것이 전부입니다!

### 6-2. 설정 패널 추가 (API Key, 시스템 프롬프트, Google Search)

왼쪽에 설정 패널, 오른쪽에 채팅 창을 배치합니다.

In [ ]:
def send_message(user_message, chat_history, api_key, system_prompt, use_search):
    """설정을 반영하여 Gemini에게 메시지를 보냅니다."""
    # API Key 확인
    if not api_key or not api_key.strip():
        chat_history.append({"role": "user", "content": user_message})
        chat_history.append({"role": "assistant", "content": "API Key를 입력해주세요."})
        return "", chat_history

    if not user_message or not user_message.strip():
        return "", chat_history

    # 클라이언트 생성
    client = genai.Client(api_key=api_key.strip())

    # 현재 날짜를 시스템 프롬프트에 추가
    today = datetime.date.today().strftime("%Y-%m-%d")
    base_instruction = f"오늘 날짜는 {today}입니다. 최신 정보를 기준으로 답변하세요."
    if system_prompt and system_prompt.strip():
        full_instruction = f"{base_instruction}\n{system_prompt.strip()}"
    else:
        full_instruction = base_instruction

    # 도구 설정
    tools = [types.Tool(google_search=types.GoogleSearch())] if use_search else None

    config = types.GenerateContentConfig(
        system_instruction=full_instruction,
        tools=tools,
    )

    # 대화 기록 변환 (Gradio -> Gemini 형식)
    contents = []
    for msg in chat_history:
        content = msg["content"]
        if isinstance(content, list):
            content = "".join(part["text"] for part in content if "text" in part)
        role = "user" if msg["role"] == "user" else "model"
        contents.append(types.Content(role=role, parts=[types.Part(text=content)]))
    contents.append(types.Content(role="user", parts=[types.Part(text=user_message)]))

    # 메시지 전송
    try:
        response = client.models.generate_content(
            model="gemini-2.5-flash-lite",
            contents=contents,
            config=config,
        )
        reply = response.text
    except Exception as e:
        reply = f"오류: {str(e)}"

    chat_history.append({"role": "user", "content": user_message})
    chat_history.append({"role": "assistant", "content": reply})
    return "", chat_history


# --- UI 구성 ---
with gr.Blocks() as demo:
    gr.Markdown("# Gemini 챗봇")

    with gr.Row():
        with gr.Column(scale=1):
            api_key = gr.Textbox(label="API Key", type="password")
            system_prompt = gr.Textbox(label="시스템 프롬프트", lines=3)
            use_search = gr.Checkbox(label="Google Search", value=True)

        with gr.Column(scale=3):
            chatbot = gr.Chatbot(height=400)
            msg = gr.Textbox(show_label=False, placeholder="메시지 입력...")
            clear = gr.Button("대화 초기화")

    msg.submit(fn=send_message,
               inputs=[msg, chatbot, api_key, system_prompt, use_search],
               outputs=[msg, chatbot])
    clear.click(fn=lambda: [], outputs=[chatbot])

demo.launch()

---
## 7단계: 대화 저장

대화 기록을 **JSON 파일**로 저장하는 기능을 추가합니다.

In [ ]:
def save_chat(chat_history):
    """대화 기록을 JSON 파일로 저장"""
    if not chat_history:
        return gr.update(value=None)

    # 파일명: chat_20250210_143052.json
    now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"chat_{now}.json"
    filepath = os.path.join("saved_chats", filename)

    os.makedirs("saved_chats", exist_ok=True)

    with open(filepath, "w", encoding="utf-8") as f:
        json.dump(chat_history, f, ensure_ascii=False, indent=2)

    return gr.update(value=filepath)


print("저장 함수 정의 완료!")
print("다음 단계에서 이 함수를 UI에 연결합니다.")

### 💡 JSON이란?

대화 기록은 이런 형태로 저장됩니다:

```json
[
  {"role": "user", "content": "안녕!"},
  {"role": "assistant", "content": "안녕하세요! 무엇을 도와드릴까요?"}
]
```

Python의 리스트/딕셔너리와 같은 구조라서 다루기 쉽습니다.

---
## 8단계: 완성 코드 🎉

지금까지 배운 모든 것을 하나로 합친 **완성 코드**입니다.  
이 코드가 바로 `app.py` 파일의 내용입니다.

In [ ]:
"""
Gemini 챗봇 - 완성 코드
Google Search로 최신 정보를 검색하는 AI 챗봇
"""
import os
import json
import datetime
from google import genai
from google.genai import types
import gradio as gr


# --- 1. 메시지 보내기 ---
def send_message(user_message, chat_history, api_key, system_prompt, use_search):
    """사용자 메시지를 Gemini에게 보내고 응답을 받습니다."""
    if not api_key or not api_key.strip():
        chat_history.append({"role": "user", "content": user_message})
        chat_history.append({"role": "assistant", "content": "API Key를 입력해주세요."})
        return "", chat_history

    if not user_message or not user_message.strip():
        return "", chat_history

    client = genai.Client(api_key=api_key.strip())

    # 현재 날짜를 시스템 프롬프트에 추가
    today = datetime.date.today().strftime("%Y-%m-%d")
    base_instruction = f"오늘 날짜는 {today}입니다. 최신 정보를 기준으로 답변하세요."
    if system_prompt and system_prompt.strip():
        full_instruction = f"{base_instruction}\n{system_prompt.strip()}"
    else:
        full_instruction = base_instruction

    tools = [types.Tool(google_search=types.GoogleSearch())] if use_search else None

    config = types.GenerateContentConfig(
        system_instruction=full_instruction,
        tools=tools,
    )

    # 대화 기록을 Gemini 형식으로 변환
    contents = []
    for msg in chat_history:
        content = msg["content"]
        if isinstance(content, list):
            content = "".join(part["text"] for part in content if "text" in part)
        role = "user" if msg["role"] == "user" else "model"
        contents.append(types.Content(role=role, parts=[types.Part(text=content)]))
    contents.append(types.Content(role="user", parts=[types.Part(text=user_message)]))

    try:
        response = client.models.generate_content(
            model="gemini-2.5-flash-lite",
            contents=contents,
            config=config,
        )
        reply = response.text
    except Exception as e:
        reply = f"오류 발생: {str(e)}"

    chat_history.append({"role": "user", "content": user_message})
    chat_history.append({"role": "assistant", "content": reply})
    return "", chat_history


# --- 2. 대화 저장 ---
def save_chat(chat_history):
    if not chat_history:
        return gr.update(value=None)
    now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filepath = os.path.join("saved_chats", f"chat_{now}.json")
    os.makedirs("saved_chats", exist_ok=True)
    with open(filepath, "w", encoding="utf-8") as f:
        json.dump(chat_history, f, ensure_ascii=False, indent=2)
    return gr.update(value=filepath)


def clear_chat():
    return []


# --- 3. UI 구성 ---
with gr.Blocks(title="Gemini 챗봇") as app:
    gr.Markdown("# Gemini 챗봇")
    gr.Markdown("Google Gemini API를 사용하는 AI 챗봇입니다.")

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("## 설정")
            api_key = gr.Textbox(label="Gemini API Key", type="password")
            system_prompt = gr.Textbox(label="시스템 프롬프트", lines=3,
                                       placeholder="예: 당신은 친절한 한국어 AI 비서입니다.")
            use_search = gr.Checkbox(label="Google Search 사용", value=True)
            gr.Markdown("---")
            gr.Markdown("## 대화 관리")
            save_btn = gr.Button("대화 저장", variant="secondary")
            save_output = gr.File(label="저장된 파일")

        with gr.Column(scale=3):
            chatbot = gr.Chatbot(label="대화", height=500)
            with gr.Row():
                msg = gr.Textbox(show_label=False, placeholder="메시지 입력...", scale=4)
                send_btn = gr.Button("전송", variant="primary", scale=1)
            clear_btn = gr.Button("대화 초기화")

    # 이벤트 연결
    inputs = [msg, chatbot, api_key, system_prompt, use_search]
    send_btn.click(fn=send_message, inputs=inputs, outputs=[msg, chatbot])
    msg.submit(fn=send_message, inputs=inputs, outputs=[msg, chatbot])
    clear_btn.click(fn=clear_chat, outputs=[chatbot])
    save_btn.click(fn=save_chat, inputs=[chatbot], outputs=[save_output])

app.launch(server_name="0.0.0.0", server_port=7860, theme=gr.themes.Soft())

---
## 9단계: Hugging Face에 배포하기 🚀

완성된 챗봇을 인터넷에 공개해봅시다!

### 배포 순서

**① Hugging Face에서 Space 만들기**
1. https://huggingface.co/new-space 접속
2. Space name: `my-chatbot`
3. SDK: **Docker** 선택
4. Create Space 클릭

**② 터미널에서 배포** (아래 셀 참고)

**③ 접속 확인**
- `https://YOUR_USERNAME-my-chatbot.hf.space`

In [ ]:
# ⬇️ 아래 명령어를 VS Code 터미널에서 실행하세요 (여기서는 실행 불가)

commands = """
# 1. HF CLI 설치 & 로그인
pip install huggingface_hub
huggingface-cli login

# 2. Space 클론
git clone https://huggingface.co/spaces/YOUR_USERNAME/my-chatbot
cd my-chatbot

# 3. 필요한 파일 복사 (app.py, requirements.txt, Dockerfile, README.md)

# 4. 배포
git add .
git commit -m "Initial deploy"
git push
"""

print(commands)

### 필요한 파일 목록

| 파일 | 용도 |
|------|------|
| `app.py` | 챗봇 메인 코드 |
| `requirements.txt` | 라이브러리 목록 |
| `Dockerfile` | 실행 환경 설정 |
| `README.md` | Space 설명 |

이 4개 파일만 있으면 배포 완료입니다! 🎉

---
## 요약

| 단계 | 배운 것 | 핵심 코드 |
|------|---------|----------|
| 2단계 | Gemini 기본 | `client.models.generate_content(...)` |
| 3단계 | 대화 모드 | `contents=[Content(...), ...]` 리스트 누적 |
| 4단계 | 시스템 프롬프트 | `system_instruction="..."` |
| 5단계 | Google Search | `tools=[types.Tool(google_search=types.GoogleSearch())]` |
| 6단계 | Gradio UI | `gr.Blocks()`, `gr.Chatbot()` |
| 7단계 | 대화 저장 | `json.dump()` |
| 9단계 | HF 배포 | `git push` |